Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [13]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16 #Number of epochs
patch_size = 5 #Kernel size
depth = 16 #Output layer after convolution
num_hidden = 64  #Number of neurons in deep layer

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))    
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    print(shape)
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print(reshape.get_shape().as_list())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 7, 7, 16]
[16, 784]
[10000, 7, 7, 16]
[10000, 784]
[10000, 7, 7, 16]
[10000, 784]


In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.737797
Minibatch accuracy: 12.5%
Validation accuracy: 9.3%
Minibatch loss at step 50: 2.075051
Minibatch accuracy: 31.2%
Validation accuracy: 43.4%
Minibatch loss at step 100: 1.645268
Minibatch accuracy: 62.5%
Validation accuracy: 65.0%
Minibatch loss at step 150: 1.639813
Minibatch accuracy: 50.0%
Validation accuracy: 69.5%
Minibatch loss at step 200: 0.655044
Minibatch accuracy: 81.2%
Validation accuracy: 76.9%
Minibatch loss at step 250: 1.005628
Minibatch accuracy: 68.8%
Validation accuracy: 76.3%
Minibatch loss at step 300: 0.825874
Minibatch accuracy: 68.8%
Validation accuracy: 77.6%
Minibatch loss at step 350: 0.774467
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 400: 0.736646
Minibatch accuracy: 75.0%
Validation accuracy: 78.5%
Minibatch loss at step 450: 0.942393
Minibatch accuracy: 62.5%
Validation accuracy: 80.2%
Minibatch loss at step 500: 1.063253
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [16]:
batch_size = 16
depth = 16
patch = 5
num_output = 64

g = tf.Graph()
with g.as_default():
    def weight_initialize(weight_size): #Initialize the weights with stddev 0.1
        weights_ready = tf.random_normal(shape=weight_size, stddev=0.1)
        return tf.Variable(weights_ready)

    def bias_initialize(bias_size): #Give a small positive bias
        bias_ready = tf.constant(0.5,shape=bias_size)
        return tf.Variable(bias_ready)

    with open('notMNIST.pickle','rb') as f:
        data = pickle.load(f)
        train_dataset = data["train_dataset"]
        train_labels = data["train_labels"]
        valid_dataset = data["valid_dataset"]
        valid_labels = data["valid_labels"]
        test_dataset = data["test_dataset"]
        test_labels = data["test_labels"]
        
    #Reformatting the shape of the data set into rank 4 as required by convolution
    
    def reformat(dataset, labels):
      dataset = dataset.reshape(
                (-1, image_size, image_size, num_channels)).astype(np.float32)
      labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
      return dataset, labels
    train_dataset, train_labels = reformat(train_dataset, train_labels)
    valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
    test_dataset, test_labels = reformat(test_dataset, test_labels)  

    #Initialize the placeholders and label variables
    tf_train_data =tf.placeholder(tf.float32,shape = (batch_size, image_size, image_size, num_channels), name = None)
    tf_labels = tf.placeholder(tf.float32,shape = (batch_size, num_labels), name = None)
    #print(tf_train_data.get_shape().as_list())
    #print(tf_labels.shape.get_shape().as_list())    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    #Conv layer1 - Here the choice of weight dimensions does not depend on input size. 
    #But based on input, kernel, padding and stride - op size is computed
    layer1_w = weight_initialize([patch,patch,num_channels, depth])
    layer1_b = bias_initialize([depth])
    #Conv layer 2- Here the choice of weight dimensions does not depend on input size. 
    #But based on input, kernel, padding and stride - op size is computed
    layer2_w = weight_initialize([patch,patch,depth, depth])
    layer2_b = bias_initialize([depth])
    #Hidden layer - hidden neurons is 64 here
    layer3_w = weight_initialize([image_size // 4 * image_size //  4 *depth , num_output])
    layer3_b = bias_initialize([num_output])
    #Readout layer
    layer4_w = weight_initialize([num_output, num_labels])
    layer4_b = bias_initialize([num_labels])
    
    def train_model(inp_data):
        #Layer1 weights 5x5x1x16
        op_1 = tf.nn.conv2d(inp_data, layer1_w, 
                            strides = [1,1,1,1], padding = 'SAME') #Convolution
        relu_1 = tf.nn.relu(op_1 + layer1_b)                       #Relu
        #By now output size is 28x28x16
        conv_1 = tf.nn.max_pool(relu_1,ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')      #Max Pooling
        #By now output is 14x14x16
        
        #Layer2 weights 5x5x1x64
        op_2 = tf.nn.conv2d(conv_1, layer2_w, 
                            strides = [1,1,1,1], padding = 'SAME') #Convolution
        relu_2 = tf.nn.relu(op_2 + layer2_b)                       #Relu
        conv_2 = tf.nn.max_pool(relu_2,ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')      #Max Pooling
        #By now output is 7x7x16 = 784x1
                
        #Layer3
        #Reshaping to suit the fully connected layer
        conv2_size = conv_2.get_shape().as_list()
        reshape_2 = tf.reshape(conv_2,[conv2_size[0],conv2_size[1]*conv2_size[2]*conv2_size[3]])
        
        op_3 = (tf.matmul(reshape_2,layer3_w) + layer3_b) #Here weights are 64x784
        relu_3 = tf.nn.relu(op_3)
        
        #Output is 64x1; l4 weights are 64x10
        #Layer4
        op_4 = (tf.matmul(relu_3,layer4_w) + layer4_b)
        
        return op_4 #size 10x1
    
    #Run the above function for a batch size and return the outputs
    logits = train_model(tf_train_data)
    
    #Loss computation for that batch size (reduce the mean for the average of those inputs for one step)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_labels))
         
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Evaluate the performance for training data (occassionally), validation and testData towards the end
    tf_train_predict = tf.nn.softmax(logits)
    tf_valid_predict = tf.nn.softmax(train_model(tf_valid_dataset))
    tf_test_predict = tf.nn.softmax(train_model(tf_test_dataset))

In [29]:
#Actual graph execution
num_steps = 1001

with tf.Session(graph = g) as sess:
  tf.initialize_all_variables().run()
  
  for step in range(num_steps):
    offset = (step)*batch_size
    train_dataset_ph = train_dataset[offset:(offset+batch_size),:,:,:]
    train_labels_ph = train_labels[offset:(offset+batch_size),:]
    #print(train_dataset_ph.shape,train_labels_ph.shape)
    feed_dict = {tf_train_data : train_dataset_ph, tf_labels : train_labels_ph}
    
    #Ouput arguments order is same as input 'fetches' arguments
    _,l,prediction = sess.run([optimizer,loss,tf_train_predict],feed_dict)
    if (step%50== 0):
      acc = accuracy (prediction,train_labels_ph)
      print("Step = %d"%(step))
      #print("Mini-batch loss at step %d : %.1f%%" % (step, l)
      print("Minibatch accuracy at step %d: %.1f%%"% (step,acc))
      
      valid_pred = sess.run(tf_valid_predict)
      valid_acc = accuracy(valid_pred,valid_labels)
      print("Validation accuracy at step %d is %.2f%%" % (step,valid_acc ))
  test_pred = sess.run(tf_test_predict)
  test_acc = accuracy(test_pred,test_labels)
  print("Final Test accuracy = %.1f%%" %test_acc)

Step = 0
Minibatch accuracy at step 0: 18.8%
Validation accuracy at step 0 is 12.38%
Step = 50
Minibatch accuracy at step 50: 37.5%
Validation accuracy at step 50 is 57.66%
Step = 100
Minibatch accuracy at step 100: 68.8%
Validation accuracy at step 100 is 67.71%
Step = 150
Minibatch accuracy at step 150: 56.2%
Validation accuracy at step 150 is 73.81%
Step = 200
Minibatch accuracy at step 200: 81.2%
Validation accuracy at step 200 is 77.19%
Step = 250
Minibatch accuracy at step 250: 68.8%
Validation accuracy at step 250 is 76.82%
Step = 300
Minibatch accuracy at step 300: 75.0%
Validation accuracy at step 300 is 80.04%
Step = 350
Minibatch accuracy at step 350: 75.0%
Validation accuracy at step 350 is 80.70%
Step = 400
Minibatch accuracy at step 400: 81.2%
Validation accuracy at step 400 is 79.73%
Step = 450
Minibatch accuracy at step 450: 68.8%
Validation accuracy at step 450 is 80.86%
Step = 500
Minibatch accuracy at step 500: 75.0%
Validation accuracy at step 500 is 81.12%
Step = 5

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [9]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

num_epochs = 10
batch_size = 64 #SGD mini batch size
depth1 = 32
depth2 = 64
patch = 5
SEED = 66478
num_output = 512

#Regularization
reg_param = 0.0005
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

g = tf.Graph()
with g.as_default():
    def weight_initialize(weight_size): #Initialize the weights with stddev 0.1
        weights_ready = tf.truncated_normal(shape=weight_size, stddev=0.1, seed=SEED)
        return tf.Variable(weights_ready)

    def bias_initialize(bias_size,val): #Give a small positive bias
        bias_ready = tf.constant(val,shape=bias_size)
        return tf.Variable(bias_ready)

    with open('notMNIST.pickle','rb') as f:
        data = pickle.load(f)
        train_dataset = data["train_dataset"]
        train_labels = data["train_labels"]
        valid_dataset = data["valid_dataset"]
        valid_labels = data["valid_labels"]
        test_dataset = data["test_dataset"]
        test_labels = data["test_labels"]
        
    #Reformatting the shape of the data set into rank 4 as required by convolution
    
    def reformat(dataset, labels):
      dataset = dataset.reshape(
                (-1, image_size, image_size, num_channels)).astype(np.float32)
      labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
      return dataset, labels
    train_dataset, train_labels = reformat(train_dataset, train_labels)
    valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
    test_dataset, test_labels = reformat(test_dataset, test_labels)  

    #Initialize the placeholders and label variables
    tf_train_data =tf.placeholder(tf.float32,shape = (batch_size, image_size, image_size, num_channels), name = None)
    tf_labels = tf.placeholder(tf.float32,shape = (batch_size, num_labels), name = None)
    
    eval_data = tf.placeholder(
      tf.float32,
      shape=(batch_size, image_size, image_size, num_channels))
    #print(tf_train_data.get_shape().as_list())
    #print(tf_labels.shape.get_shape().as_list())    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    #Conv layer1 - Here the choice of weight dimensions does not depend on input size. 
    #But based on input, kernel, padding and stride - op size is computed
    layer1_w = weight_initialize([patch,patch,num_channels, depth1])
    layer1_b = bias_initialize([depth1],0.1)
    #Conv layer 2- Here the choice of weight dimensions does not depend on input size. 
    #But based on input, kernel, padding and stride - op size is computed
    layer2_w = weight_initialize([patch,patch,depth1, depth2])
    layer2_b = bias_initialize([depth2],0.1)
    #Hidden layer - hidden neurons is num_output here
    layer3_w = weight_initialize([image_size // 4 * image_size //  4 *depth2 , num_output])
    layer3_b = bias_initialize([num_output],0.1)
    #Readout layer
    layer4_w = weight_initialize([num_output, num_labels])
    layer4_b = bias_initialize([num_labels],0.1)
    #Dropput_Keep probability
    keep_prob = tf.placeholder(tf.float32)  
    #Learning rate
    learn_rate_init = tf.placeholder(tf.float32)
    train_size = train_labels.shape[0]
    def train_model(inp_data, train = False):
        #Layer1 weights 5x5x1x16
        op_1 = tf.nn.conv2d(inp_data, layer1_w, 
                            strides = [1,1,1,1], padding = 'SAME') #Convolution
        relu_1 = tf.nn.relu(op_1 + layer1_b)                       #Relu
        #By now output size is 28x28x16
        conv_1 = tf.nn.max_pool(relu_1,ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')      #Max Pooling
        #By now output is 14x14x16
        
        #Layer2 weights 5x5x1x64
        op_2 = tf.nn.conv2d(conv_1, layer2_w, 
                            strides = [1,1,1,1], padding = 'SAME') #Convolution
        relu_2 = tf.nn.relu(op_2 + layer2_b)                       #Relu
        conv_2 = tf.nn.max_pool(relu_2,ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')      #Max Pooling
        #By now output is 7x7x16 = 784x1
                
        #Layer3
        #Reshaping to suit the fully connected layer
        conv2_size = conv_2.get_shape().as_list()
        reshape_2 = tf.reshape(conv_2,[conv2_size[0],conv2_size[1]*conv2_size[2]*conv2_size[3]])
            
        op_3 = (tf.matmul(reshape_2,layer3_w) + layer3_b) #Here weights are 64x784
        relu_3 = tf.nn.relu(op_3)
        #Adding dropout only if training
        if train:
         relu_3 = tf.nn.dropout(relu_3,keep_prob)
        
        #Output is 64x1; l4 weights are 64x10
        #Layer4
        op_4 = (tf.matmul(relu_3,layer4_w) + layer4_b)
        
        return op_4 #size 10x1
    
    #Run the above function for a batch size and return the outputs
    logits = train_model(tf_train_data, train = True)
    
    #Loss computation for that batch size (reduce the mean for the average of those inputs for one step)
    #Add regularization for only fully connexted weights
    regularizers = (tf.nn.l2_loss(layer3_w) + tf.nn.l2_loss(layer3_b)
                           + tf.nn.l2_loss(layer4_w) + tf.nn.l2_loss(layer4_b))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_labels))
    loss+= reg_param * regularizers
    
    #Optimizer with decaying learning rate
    # Optimizer: set up a variable that's incremented once per batch and controls the learning rate decay.
    batch = tf.Variable(0, dtype=tf.float32)
    # Decay once per epoch, using an exponential schedule starting at 0.01.
    learning_rate = tf.train.exponential_decay(
      learn_rate_init,                # Base learning rate.
      batch * batch_size,  # Current index into the dataset.
      train_size,          # Decay step.
      0.95,                # Decay rate.
      staircase=True)
    #Use Gradient Descent optimizer
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    # Use simple momentum for the optimization.
    optimizer = tf.train.MomentumOptimizer(learning_rate,0.9).minimize(loss,global_step=batch)
    
    #Evaluate the performance for training data (occassionally), validation and testData towards the end
    tf_train_predict = tf.nn.softmax(logits)
    # Predictions for the test and validation, which we'll compute less often.
    eval_prediction = tf.nn.softmax(train_model(eval_data))#This will predict for small batches of eval data
    
  # Small utility function to evaluate a dataset by feeding batches of data to
  # {eval_data} and pulling the results from {eval_predictions}.
  # Saves memory and enables this to run on smaller GPUs.
    def eval_in_batches(data, sess):
    #Get all predictions for a dataset by running it in small batches."""
        size = data.shape[0]
        if size < batch_size:
          raise ValueError("batch size for evals larger than dataset: %d" % size)
        predictions = np.ndarray(shape=(size, num_labels), dtype=np.float32)
        for begin in xrange(0, size, batch_size):
          end = begin + batch_size
          if end <= size:
            predictions[begin:end, :] = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[begin:end, ...]})
          else:
            batch_predictions = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[-batch_size:, ...]})
            predictions[begin:, :] = batch_predictions[begin - size:, :]
        return predictions
    
    tf_valid_predict = tf.nn.softmax(train_model(tf_valid_dataset)) #This will predict for whole eval data at once
    tf_test_predict = tf.nn.softmax(train_model(tf_test_dataset))

In [10]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph = g) as sess:
    tf.initialize_all_variables().run()
    for step in range(int(num_epochs*train_size // batch_size)):
        offset = (step*batch_size) % (train_size - batch_size) #To ensure the offset doesnt exceed train_size
        train_dataset_ph = train_dataset[offset:(offset+batch_size),:,:,:]
        train_labels_ph = train_labels[offset:(offset+batch_size),:]
        #print(train_dataset_ph.shape,train_labels_ph.shape)
        feed_dict = {tf_train_data : train_dataset_ph, tf_labels : train_labels_ph,
                    keep_prob : 0.5, learn_rate_init : 0.01 }

        #Ouput arguments order is same as input 'fetches' arguments
        _,l,prediction,learn_rate = sess.run([optimizer,loss,tf_train_predict,learning_rate],feed_dict)
        
        if (step%300== 0):
          acc = accuracy (prediction,train_labels_ph)
          print("Step = %d for epoch = %.2f"%(step, float(step)*batch_size / train_size))
          print("learning rate = %.6f%%, Mini-batch loss: %.1f%%, Minibatch accuracy: %.1f%%" % (learn_rate,l,acc))
          #This is for big chunk of validation data
          #valid_pred = sess.run(tf_valid_predict)
          #valid_acc = accuracy(valid_pred,valid_labels)
          #print("Validation accuracy at step %d is %.2f%%" % (step,valid_acc ))
        #This is for small chunks of validation data
          print('Validation accuracy: %.1f%%' % accuracy(
          eval_in_batches(valid_dataset, sess), valid_labels))
    
    test_pred = sess.run(tf_test_predict)
    test_acc = accuracy(test_pred,test_labels)
    print("Final Test accuracy = %.1f%%" %test_acc)

Step = 0 for epoch = 0.00
learning rate = 0.010000%, Mini-batch loss: 13.2%, Minibatch accuracy: 12.5%
Validation accuracy: 15.8%
Step = 300 for epoch = 0.10
learning rate = 0.010000%, Mini-batch loss: 3.7%, Minibatch accuracy: 82.8%
Validation accuracy: 83.5%
Step = 600 for epoch = 0.19
learning rate = 0.010000%, Mini-batch loss: 3.5%, Minibatch accuracy: 82.8%
Validation accuracy: 85.4%
Step = 900 for epoch = 0.29
learning rate = 0.010000%, Mini-batch loss: 3.4%, Minibatch accuracy: 87.5%
Validation accuracy: 86.9%
Step = 1200 for epoch = 0.38
learning rate = 0.010000%, Mini-batch loss: 3.2%, Minibatch accuracy: 85.9%
Validation accuracy: 87.2%
Step = 1500 for epoch = 0.48
learning rate = 0.010000%, Mini-batch loss: 3.1%, Minibatch accuracy: 89.1%
Validation accuracy: 87.5%
Step = 1800 for epoch = 0.58
learning rate = 0.010000%, Mini-batch loss: 3.1%, Minibatch accuracy: 85.9%
Validation accuracy: 88.0%
Step = 2100 for epoch = 0.67
learning rate = 0.010000%, Mini-batch loss: 3.0%, Mi